In [1]:
from secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key

In [2]:
!pip install -U langchain-openai
!pip install wikipedia


In [8]:
# !which openai
from openai import OpenAI
client = OpenAI()


completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='In the realm of code, a magic unfolds,\nWhere recursion weaves its stories untold.\nA function calling itself, dancing in loops,\nA symphony of patterns, where elegance swoops.\n\nLike a mirror reflecting into the night,\nEach invocation a shimmering light.\nDividing problems, small and discrete,\nConquering complexities, oh so neat.\n\nInfinite beauty in a finite space,\nA cascade of calls, with grace and embrace.\nA journey within, echoing deep,\nA recursive dream, in slumber we keep.\n\nSo embrace the recursion, fearless and bold,\nIn the realm of code, let your story be told.\nFor in loops that loop back unto themselves,\nLies the magic of coding, where wonder dwells.', role='assistant', function_call=None, tool_calls=None)


In [9]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.6)
name = llm("I want to open a nepali restaurant for Nepali food. Suggest me a fancy name for it.")
print(name)

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




Himalayan Spice Bistro


/opt/anaconda3/bin/python


Name: langchain
Version: 0.1.16
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: aiohttp, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [12]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this"
    )
prompt_template_name.format(cuisine="Nepali")


'I want to open a restaurant for Nepali food. Suggest a fancy name for this'

In [13]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Indian")


/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n"Spice Palace"'

In [14]:
# Simple sequential chain

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this"
    )
name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it as a comma-separated list."
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [15]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])
response = chain.run("Nepali")
print(response)



1. Chicken Tikka Masala
2. Vegetable Samosas
3. Lamb Vindaloo
4. Saag Paneer
5. Tandoori Chicken
6. Chana Masala
7. Aloo Gobi
8. Naan Bread
9. Mango Lassi
10. Vegetable Biryani


In [45]:
# Sequential Chain

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this"
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it as a comma-separated list."
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")



In [19]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']

)
chain({'cuisine':'Nepali'})


/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'cuisine': 'Nepali',
 'restaurant_name': '\n\n"Everest Eats: A Taste of Nepal"',
 'menu_items': '\n\n1. Momo dumplings\n2. Dal bhat (lentil soup and rice)\n3. Chicken or vegetable curry \n4. Gorkhali lamb \n5. Tandoori chicken \n6. Aloo gobi (potato and cauliflower curry) \n7. Saag paneer (spinach and cheese curry) \n8. Samosas \n9. Thukpa (Nepalese noodle soup) \n10. Chana masala (chickpea curry) \n11. Tarkari (mixed vegetable curry) \n12. Nepali-style fried rice \n13. Chicken or vegetable momo soup \n14. Kheer (rice pudding) \n15. Lassi (yogurt-based drink)'}

In [24]:

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
agent.run("When was Elon musk born? What is his age right now today?")




> Entering new AgentExecutor chain...
 you should always think about what to do
Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk (; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of The Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be US$196 billion. 
A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania an

'Elon Musk was born on June 28, 1971 and is currently 50 years old.'

In [39]:
from secret import serpapi_key
os.environ['SERPAPI_API_KEY'] = serpapi_key

In [41]:
pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32003 sha256=83407d3f0dcc91ed9ca1b43a8841917deac806b1e7c9074ed8be52a62e22d261
  Stored in directory: /Users/shivakhatri/Library/Caches/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results
Note: you may need to restart the kernel to use updated packages.


In [42]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run("What is the current GDP of US and add 1 to it?")




> Entering new AgentExecutor chain...
 I will need to search for the current GDP of the US and then use a calculator to add 1 to it.
Action: Search
Action Input: "US GDP"
Observation: [{'title': 'GDP Gain in First Quarter Revised Downward in U.S.', 'link': 'https://www.nytimes.com/2024/05/30/business/economy/us-economic-growth-first-quarter.html', 'source': 'The New York Times', 'date': '2 days ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRNiHVcyYGRlR8Qeq84CISAfhBl6TBluEdoXFqsrHGLv1o9GmiVrQlmzEf0yQ&usqp=CAI&s=10'}, {'title': 'US GDP Q1 2024: Economy Grew at Softer Pace as Spending, Inflation Marked Down', 'link': 'https://www.bloomberg.com/news/articles/2024-05-30/us-gdp-grew-at-softer-pace-as-spending-inflation-marked-down', 'source': 'Bloomberg', 'date': '2 days ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ5TSV9g8dMkQLcVFt5Jgi4iL-wNvPo7QObJ6ieavXC_u7zGHIzFLATD5c&usqp=CAI&s'}, {'title': 'GDP: US economy grew at a slower pace

'The current GDP of the US is 2.3.'

In [44]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



"Cantina de Lujo" (which translates to "Luxury Cantina") 


In [46]:
chain.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Mexican'), AIMessage(content='\n\n"Cantina de Lujo" (which translates to "Luxury Cantina") ')]))

In [48]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"Cantina de Lujo" (which translates to "Luxury Cantina") 


In [50]:
# Unlimited Memory
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
# convo.prompt

convo.run("What is the capital city of Nepal")

" The capital city of Nepal is Kathmandu. It is located in the Kathmandu Valley, surrounded by four major mountains: Shivapuri, Phulchowki, Nagarjun, and Chandragiri. Kathmandu is the largest city in Nepal and serves as the country's economic, cultural, and political center. It is also home to several important historical and religious sites, including the Pashupatinath Temple, Swayambhunath Stupa, and Boudhanath Stupa."

In [43]:
# Limited Memory determinded by k
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k = 1)

convo = ConversationChain(
    llm = OpenAI(temperature=0.6),
    memory=memory
)
convo.run("Who was the first king of Nepal")


NameError: name 'ConversationChain' is not defined